In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score
from matplotlib import pyplot

from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adagrad, Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Explainer NN
![title](explainer.jpeg)

In [2]:
def create_explainer_model():
    # Lets build the Model
    model = Sequential()
    # No of Input will be == (total number of train examples , 8)
    # where 8 = feature
    model.add(Input(shape=(4,)))

    # Hidden Layer 1
    model.add(Dense(units=3,activation='relu'))
    # Hidden Layer 2
    model.add(Dense(units=3,activation='relu'))
    # Hidden Layer 3
    model.add(Dense(units=3,activation='relu'))
    # Output Layer - this is a binary classification
    model.add(Dense(units=2,activation='softmax'))
    return model

model = create_explainer_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 15        
                                                                 
 dense_1 (Dense)             (None, 3)                 12        
                                                                 
 dense_2 (Dense)             (None, 3)                 12        
                                                                 
 dense_3 (Dense)             (None, 2)                 8         
                                                                 
Total params: 47
Trainable params: 47
Non-trainable params: 0
_________________________________________________________________


2023-02-15 21:04:16.023450: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from numpy.random import default_rng

def create_data(num_samples,num_features, num_class=1):
    if(num_class == 0):
        features = default_rng(42).random((num_samples,num_features))
        targets = default_rng(4242).random((num_samples,1))
    else:
        features = default_rng(42).random((num_samples,num_features))
        targets = default_rng(42).choice(np.arange(0, num_class,1, dtype=int),(num_samples,1))
    return features,targets


In [4]:
features, target = create_data(num_samples=10,num_features=4,num_class=2)
print(features.shape)
print(target.shape)

(10, 4)
(10, 1)


In [5]:
# Lets see step by step the calculation
optimizer = Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer, loss=loss_fn)

# Open a GradientTape.
with tf.GradientTape() as tape:
    # Forward pass.
    probab = model(features)
    print('logits (though this is softmax proba output)=',probab)
    # Loss value for this batch.
    loss_value = loss_fn(target, probab)
    print('')
    print('loss_value = ',loss_value)
print('')
# Weights after 1 forward pass 
print('get_weights = ',model.get_weights())
print('')

#BACKPROP - Gradient Compute
# Get gradients of loss wrt the weights.
gradients = tape.gradient(loss_value, model.trainable_weights)
print('gradients of loss wrt the weight = ',gradients)

# Update the weights of the model.
optimizer.apply_gradients(zip(gradients, model.trainable_weights))
print('')
print('updated weights=',optimizer.weights)

logits (though this is softmax proba output)= tf.Tensor(
[[0.5031874  0.4968126 ]
 [0.4359087  0.5640913 ]
 [0.45834255 0.5416574 ]
 [0.48702827 0.51297176]
 [0.50528365 0.4947163 ]
 [0.50568306 0.49431694]
 [0.5        0.5       ]
 [0.46542495 0.53457505]
 [0.5015132  0.49848682]
 [0.45311567 0.5468843 ]], shape=(10, 2), dtype=float32)

loss_value =  tf.Tensor(0.6780264, shape=(), dtype=float32)

get_weights =  [array([[ 0.25020564, -0.26055712, -0.4651516 ],
       [-0.53109527, -0.02203929,  0.7301979 ],
       [-0.62593997,  0.21514106, -0.5917058 ],
       [-0.4054922 ,  0.14544344,  0.20910561]], dtype=float32), array([0., 0., 0.], dtype=float32), array([[ 0.87567306, -0.26388955, -0.43276072],
       [-0.07409286,  0.19441581,  0.7062895 ],
       [ 0.6598475 ,  0.17842174, -0.8675635 ]], dtype=float32), array([0., 0., 0.], dtype=float32), array([[ 0.83724785, -0.39285994,  0.7213776 ],
       [ 0.16100192,  0.33541274,  0.709605  ],
       [ 0.7494116 ,  0.15365767, -0.15588093

In [ ]:
import pprint
model_json = model.to_json()
print(model_json)

{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 4], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_1"}}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "dtype": "float32", "units": 3, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 3, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularize

In [6]:
model.save('explainer_model.h5')

In [7]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard
import tensorboard
from datetime import datetime
tensorboard.__version__


'2.9.1'

In [8]:
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

f, t = create_data(num_samples=100,num_features=4,num_class=2)
m = create_explainer_model()
m.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
# Train the model.
model.fit(
    f,
    t, 
    batch_size=20,
    epochs=5, 
    callbacks=[tensorboard_callback])

Epoch 1/5
5/5 [==============================] - 0s 3ms/step - loss: 0.6890
Epoch 2/5
5/5 [==============================] - 0s 3ms/step - loss: 0.6885
Epoch 3/5
5/5 [==============================] - 0s 3ms/step - loss: 0.6882
Epoch 4/5
5/5 [==============================] - 0s 2ms/step - loss: 0.6878
Epoch 5/5
5/5 [==============================] - 0s 2ms/step - loss: 0.6877


In [10]:
%tensorboard --logdir logs

# !tensorboard dev upload \
#   --logdir logs \
#   --name "Sample op-level graph" \
#   --one_shot

Reusing TensorBoard on port 6006 (pid 52698), started 0:00:20 ago. (Use '!kill 52698' to kill it.)